In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn  
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, make_scorer
from keras import preprocessing as pre
from keras import Sequential
from keras import layers as lay
from keras import layers, metrics, optimizers, models, losses, utils
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np  
from PIL import Image
import glob
import tensorflow as tf
from tensorflow  import gpu



In [2]:
sklearn.set_config(transform_output="pandas")

In [3]:

dir = r"C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Data\LEGO brick images v1"

In [4]:
df,df_val= pre.image_dataset_from_directory(
    directory = dir,
    labels='inferred', 
    label_mode='categorical',
    color_mode="grayscale",
    image_size=(200, 200),
    batch_size = 128,
    shuffle=True,
    seed=42,
    interpolation="bilinear",
    validation_split=0.2,
    subset='both'
)

Found 6379 files belonging to 16 classes.
Using 5104 files for training.
Using 1275 files for validation.


In [5]:
model = Sequential(
       layers=(
           
           lay.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(200, 200, 1)),
           lay.MaxPooling2D(2,2),

           lay.Dense(64,activation='relu'),
           lay.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),

           lay.Dropout(0.2),
           lay.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),
           
           lay.Dense(32, activation='relu'),
           lay.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),
           
           lay.Flatten(),
           lay.Dense(16, activation='softmax')
           
           )
       )



c:\Users\bruno\OneDrive\Documenti\Deep_learning_project\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy'])


In [7]:
epoche = 10

In [8]:

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 99, 99, 64)     │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23, 23, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 21, 21, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │       204,816 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 282,096 (1.08 MB)

 Trainable params: 282,096 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Stampa le informazioni sulle GPU disponibili
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [10]:
# Verifica l'accelerazione GPU per TensorFlow
print("GPU acceleration available:" , tf.config.experimental.list_physical_devices('GPU'))


GPU acceleration available: []


In [11]:
# Verifica se TensorFlow sta utilizzando la GPU
print("TensorFlow is using GPU:", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
TensorFlow is using GPU: False


In [12]:
history = model.fit(df,epochs=epoche,validation_data=df_val,batch_size=32)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.2512 - loss: 3.3651 - val_accuracy: 0.6463 - val_loss: 0.9002
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.7137 - loss: 0.7662 - val_accuracy: 0.7686 - val_loss: 0.5950
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8200 - loss: 0.4920 - val_accuracy: 0.7859 - val_loss: 0.5538
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8598 - loss: 0.3681 - val_accuracy: 0.8290 - val_loss: 0.4604
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.8932 - loss: 0.2947 - val_accuracy: 0.8447 - val_loss: 0.4356
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.9186 - loss: 0.2253 - val_accuracy: 0.8518 - val_loss: 0.4391
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.9255 - loss: 0.1909 - val_accuracy: 0.8776 - val_loss: 0.3571
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.9493 - loss: 0.1427 - val_accuracy: 0.8722 - val_loss:

In [13]:
model.export(filepath=r'C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Testing')

INFO:tensorflow:Assets written to: C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Testing\assets


INFO:tensorflow:Assets written to: C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Testing\assets


Saved artifact at 'C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Testing'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 16), dtype=tf.float32, name=None)
Captures:
  2462931294480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931293712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931294672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931295440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931294288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931295824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931294864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931295632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931295248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2462931297360: TensorSpec(shape=(), dtype=tf.resource,